In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D

In [154]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :4]

        past_candles = []
        #for z in range(1, how_many_past_candles+1):
        #  past_candles.append(data[i-z][3])
        #past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append(data[i+z][3])
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > data[i-1][3]:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        del rows
        del past_candles
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                           scale_padding=0.2, returnfig=True, savefig=address, tight_layout=True,figsize =(3,2))
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [155]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd"]
download_data(symbols,"max","1d")

Symbols : 1500
Data Folder Removed
 


In [156]:
how_many_future_candles = 3
how_many_past_candles = 3
each_row_past = 100

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 3
Processing File:
File Number 1: 10/1604 20/1604 30/1604 40/1604 50/1604 60/1604 70/1604 80/1604 90/1604 100/1604 110/1604 120/1604 130/1604 140/1604 150/1604 160/1604  
170/1604 180/1604 190/1604 200/1604 210/1604 220/1604 230/1604 240/1604 250/1604 260/1604 270/1604 280/1604 290/1604 300/1604 310/1604 320/1604  
330/1604 340/1604 350/1604 360/1604 370/1604 380/1604 390/1604 400/1604 410/1604 420/1604 430/1604 440/1604 450/1604 460/1604 470/1604 480/1604  
490/1604 500/1604 510/1604 520/1604 530/1604 540/1604 550/1604 560/1604 570/1604 580/1604 590/1604 600/1604 610/1604 620/1604 630/1604 640/1604  
650/1604 660/1604 670/1604 680/1604 690/1604 700/1604 710/1604 720/1604 730/1604 740/1604 750/1604 760/1604 770/1604 780/1604 790/1604 800/1604  
810/1604 820/1604 830/1604 840/1604 850/1604 860/1604 870/1604 880/1604 890/1604 900/1604 910/1604 920/1604 930/1604 940/1604 950/1604 960/1604  
970/1604 980/1604 990/1604 1000/1604 1010/1604 1020/1604 1030/1604 1040/1604 1050/1

5961

In [ ]:
#folder_name = '083812'
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)
    dim = (75, 75)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255.0
data.shape

In [170]:
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.1,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

xTrain : 5364 \ xTest : 597
yn: 3188 nn: 2773


In [171]:
model = Sequential()

model.add(Conv2D(64,  (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,  (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(150,  (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_125 (Conv2D)         (None, 73, 73, 64)        640       
                                                                 
 max_pooling2d_116 (MaxPooli  (None, 36, 36, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_126 (Conv2D)         (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_117 (MaxPooli  (None, 17, 17, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_127 (Conv2D)         (None, 15, 15, 150)       86550     
                                                                 
 max_pooling2d_118 (MaxPooli  (None, 7, 7, 150)      

In [172]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(xTrain,yTrain,batch_size=32,epochs=25,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)


Epoch 1/25
168/168 [==============================] - 4s 18ms/step - loss: 0.6952 - accuracy: 0.5393 - val_loss: 0.6927 - val_accuracy: 0.5176
Epoch 2/25
168/168 [==============================] - 2s 15ms/step - loss: 0.6911 - accuracy: 0.5319 - val_loss: 0.6941 - val_accuracy: 0.5176
Epoch 3/25
168/168 [==============================] - 2s 14ms/step - loss: 0.6909 - accuracy: 0.5367 - val_loss: 0.6928 - val_accuracy: 0.5176
Epoch 4/25
168/168 [==============================] - 2s 14ms/step - loss: 0.6905 - accuracy: 0.5367 - val_loss: 0.6932 - val_accuracy: 0.5176
Epoch 5/25
168/168 [==============================] - 2s 14ms/step - loss: 0.6908 - accuracy: 0.5367 - val_loss: 0.6925 - val_accuracy: 0.5176
Epoch 6/25
168/168 [==============================] - 2s 14ms/step - loss: 0.6909 - accuracy: 0.5367 - val_loss: 0.6930 - val_accuracy: 0.5176
Epoch 7/25
168/168 [==============================] - 2s 14ms/step - loss: 0.6889 - accuracy: 0.5406 - val_loss: 0.6922 - val_accuracy: 0.5176